<a href="https://colab.research.google.com/github/Zumo09/Feedback-Prize/blob/main/prova_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone the entire repo.
!git clone -l -s git://github.com/Zumo09/Feedback-Prize.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 16151, done.
remote: Counting objects: 100% (16151/16151), done.
remote: Compressing objects: 100% (16019/16019), done.
remote: Total 16151 (delta 295), reused 15884 (delta 108), pack-reused 0
Receiving objects: 100% (16151/16151), 143.47 MiB | 15.22 MiB/s, done.
Resolving deltas: 100% (295/295), done.
Checking out files: 100% (15640/15640), done.
/content/cloned-repo
class_correlation.ipynb  engine.py  main.py    prova.ipynb	 util
csv			 input	    models     prova_main.ipynb
datasets		 LICENSE    old_utils  README.md


In [2]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 8.7 MB/s 
     |████████████████████████████████| 596 kB 60.4 MB/s 
     |████████████████████████████████| 6.8 MB 51.3 MB/s 
     |████████████████████████████████| 67 kB 6.5 MB/s 
     |████████████████████████████████| 895 kB 62.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
%reload_ext autoreload
%autoreload 2

In [1]:
import torch

In [2]:
from argparse import Namespace

args = Namespace(
    num_queries=20,
    hidden_dim=10,
    random_state=42,
    preprocessing=True,
    test_size=0.2,
    batch_size = 5,
    input_path="./input/feedback-prize-2021/",
    device="cuda" if torch.cuda.is_available() else "cpu",
    set_cost_class=1,
    set_cost_bbox=1,
    set_cost_giou=1,
    bbox_loss_coef=0.5,
    giou_loss_coef=0.1,
    overlap_loss_coef=0.2,
    eos_coef=0.1,
    num_workers=2,
    start_epoch=0,
    epochs=10,
    lr=0.001,
    weight_decay=0.1,
    lr_drop=1,
    clip_max_norm=0,
    seed=42,
    output_dir="./outputs",
    resume=None,
    frozen_weights=None,
    eval=False,
    train_transformer=False
)

device = torch.device(args.device)

In [4]:
num_classes=7

In [5]:
from datasets import build_fdb_data, collate_fn

dataset_train, dataset_val, postprocessor, num_classes = build_fdb_data(args)

100%|██████████| 15594/15594 [00:41<00:00, 374.08it/s] 


In [5]:
from models import build_models

tokenizer, model, criterion = build_models(num_classes, args)
model.to(device)

n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("number of params:", n_parameters)

Some weights of the model checkpoint at allenai/led-base-16384 were not used when initializing LEDModel: ['lm_head.weight', 'final_logits_bias']
- This IS expected if you are initializing LEDModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LEDModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


number of params: 106134


In [7]:
import time
import datetime
from torch.utils.data import DataLoader

optimizer = torch.optim.AdamW(
    model.parameters(), lr=args.lr, weight_decay=args.weight_decay
)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, args.lr_drop)

data_loader_train = DataLoader(
    dataset_train,
    shuffle=True,
    batch_size=args.batch_size,
    collate_fn=collate_fn,
    num_workers=args.num_workers,
)
data_loader_val = DataLoader(
    dataset_val,
    shuffle=False,
    batch_size=args.batch_size,
    drop_last=False,
    collate_fn=collate_fn,
    num_workers=args.num_workers,
)

In [ ]:
from engine import train_one_epoch, evaluate

print("Start training")
start_time = time.time()
for epoch in range(args.start_epoch, args.epochs):
    train_one_epoch(
        tokenizer=tokenizer,
        model=model,
        criterion=criterion,
        data_loader=data_loader_train,
        optimizer=optimizer,
        device=device,
        epoch=epoch,
        max_norm=args.clip_max_norm,
    )
    lr_scheduler.step()

    postprocessor.reset_results()
    evaluate(
        tokenizer=tokenizer,
        model=model,
        criterion=criterion,
        postprocessor=postprocessor,
        data_loader=data_loader_val,
        epoch=epoch,
        device=device,
        tag="Validation",
    )
total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print("Training time {}".format(total_time_str))

In [ ]:
from engine import train_one_epoch, evaluate
postprocessor.reset_results()
report = evaluate(
    tokenizer=tokenizer,
    model=model,
    criterion=criterion,
    postprocessor=postprocessor,
    data_loader=data_loader_val,
    epoch=0,
    device=device,
    tag="Validation",
)
print(report.to_string())